Given: A directed graph that contains an Eulerian path, where the graph is given in the form of an adjacency list.

In [1]:
def FormatGraph(graph):
  graph_dict = {}
  for edge in graph:
    edge = edge.split(' -> ')
    graph_dict.setdefault(int(edge[0]),list())
  for edge in graph:
    edge = edge.split(' -> ')
    for i in range(len(edge[1].split(','))):
      graph_dict[int(edge[0])].append(int(edge[1].split(',')[i]))
  return graph_dict

In [2]:
def VisitedEdges(graph_dict):
  visited_edges_dict = {}
  for key,values_list in graph_dict.items():
    for value in values_list:
      visited_edges_dict.setdefault((key,value), 0)
  return visited_edges_dict

In [3]:
def NodeInDegree(graph_dict,node):
  node_indegree = 0
  for adjacent_nodes_list in graph_dict.values():
    if node in adjacent_nodes_list:
      node_indegree = node_indegree + 1
  return node_indegree

In [22]:
def NodeOutDegree(graph_dict,node):
  if node in graph_dict.keys():
    return len(graph_dict[node])
  else: #node has no adjacent nodes
    return 0

In [5]:
from random import randint, randrange

In [6]:
def EulerianCycle(graph_dict):
  visited_edges_dict = VisitedEdges(graph_dict)
  starting_node = randint(min(graph_dict.keys()), max(graph_dict.keys()))
  cycle = [starting_node]
  while sum(visited_edges_dict.values()) < len(visited_edges_dict): #repeat until Eulerian cycle is found --> input is an Eulerian directed graph --> Eulerian cycle can always be found
    #while loop entered --> sum(visited_edges_dict.values()) < len(visited_edges_dict) --> cycle smaller than Eulerian cycle is being formed
    possible_adjacent_nodes = [key[1] for key in visited_edges_dict.keys() if key[0] == cycle[len(cycle)-1] and visited_edges_dict[key] == 0]
    if len(possible_adjacent_nodes) == 0 and sum(visited_edges_dict.values()) < len(visited_edges_dict): #cycle smaller than Eulerian cycle completed as we got stuck at starting node --> all edges are not visited
      #possible_starting_nodes_list = [node for node in cycle if NodeOutDegree(graph_dict,node) >= NodeOutDegree(graph_dict,cycle[0])] #no, thsi way we are choosing node regardless of the number of times it appeared in cycle
      #possible_starting_nodes_list = [visited_edge[0] for node in cycle for visited_edge in visited_edges_dict.keys() if visited_edge[0] == node and visited_edges_dict[visited_edge] == 1] --> this caused efficiency problems
      possible_starting_nodes_list = [node for node in cycle if NodeOutDegree(graph_dict,node) > cycle.count(node)] #if NodeOutDegree(node) > number of times node occurs in cycle then there are unused outgoing edges, every occurence measn that one outgoing edge is used 
      starting_node = possible_starting_nodes_list[randrange(0,len(possible_starting_nodes_list))] #randomly choose new starting node among nodes with higher NodeOutDegree than previous starting node
      cycle = cycle[cycle.index(starting_node):len(cycle)] + cycle[1:cycle.index(starting_node)+1] #construct new_cycle using previous cycle
    else: #len(possible_adjacent_nodes) > 1 and sum(visited_edges_dict.values()) < len(visited_edges_dict) --> cycle is not finished yet
      if len(possible_adjacent_nodes) == 1:
        next_node = possible_adjacent_nodes[0]
        visited_edges_dict[(cycle[len(cycle)-1], next_node)] = 1
        cycle.append(next_node)
      else:
        next_node = possible_adjacent_nodes[randint(0,len(possible_adjacent_nodes)-1)]
        visited_edges_dict[(cycle[len(cycle)-1], next_node)] = 1
        cycle.append(next_node)
  return cycle

In [7]:
def FindUnbalancedNodes(graph_dict):
  unbalanced_nodes = []
  for node in graph_dict.keys():
    if NodeInDegree(graph_dict,node) != NodeOutDegree(graph_dict,node):
      unbalanced_nodes.append(node)
  for adjacent_nodes_list in graph_dict.values():
    for adjacent_node in adjacent_nodes_list:
      if adjacent_node not in graph_dict.keys():
        unbalanced_nodes.append(adjacent_node)
  return unbalanced_nodes

U grafu sa Eulerovim putom imamo 2 nebalansirana čvora --> ne možemo imati samo jedan nebalansirani čvor jer njegovim balansiranjem (dodavanjem ulazne ili izlazne veze) neki drugi čvor ćemo učinit nebalansiranim. Dakle, ne možemo imati neparan broj nebalansiranih čvorova.

Thus, a nearly balanced graph has an Eulerian path if and only if
adding an edge between its unbalanced nodes makes the graph balanced and strongly
connected. --> dakle, nebalansirani graf ima Eulerov put ako i samo ako dodavanjem veze između njegovih nebalansiranih čvorova graf postane balansiran i čvrsto povezan.

A path that traverses each edge of a graph exactly once (but does not necessarily return to its starting node) is called an Eulerian path.

In [8]:
def UnbalancedNodesOrder(graph_dict,unbalanced_nodes):
  ordered_unbalanced_nodes = [unbalanced_nodes[0]]
  if NodeInDegree(graph_dict, unbalanced_nodes[1]) < NodeOutDegree(graph_dict, unbalanced_nodes[1]): #node lacks one incoming edge --> node is starting node in Eulerian path
    ordered_unbalanced_nodes.insert(0, unbalanced_nodes[1])
  else: #NodeInDegree(graph_dict, unbalanced_nodes[1]) > NodeOutDegree(graph_dict, unbalanced_nodes[1]) --> node lacks one outgoing edge --> node is ending node in Eulerian path
    ordered_unbalanced_nodes.insert(1, unbalanced_nodes[1])
  return ordered_unbalanced_nodes

In [9]:
def BalanceUnbalancedNodes(graph_dict, ordered_unbalanced_nodes):
  #ordered_unbalanced_nodes = [starting_node, ending_node]
  graph_dict.update({ordered_unbalanced_nodes[1]:[ordered_unbalanced_nodes[0]]})
  return graph_dict

In [ ]:
import numpy as np

Postoji samo jedan Eulerov put u grafu.

In [268]:
def FindEulerianPathInEulerianCycle(ordered_unbalanced_nodes,eulerian_cycle):
  #ordered_unbalanced_nodes = [starting_node,ending_node]
  eulerian_cycle = np.array(eulerian_cycle)
  eulerian_path = []
  eulerian_path_start_indices = list(np.where(eulerian_cycle == ordered_unbalanced_nodes[0])[0])
  eulerian_path_end_indices = list(np.where(eulerian_cycle == ordered_unbalanced_nodes[1])[0])
  eulerian_cycle = list(eulerian_cycle)
  for start_index in eulerian_path_start_indices:
    for end_index in eulerian_path_end_indices:
      if end_index < start_index:
        if (len(eulerian_cycle) - 1 - start_index + 1) + (end_index - 0 + 1) == len(eulerian_cycle):
          return eulerian_cycle[start_index:len(eulerian_cycle)] + eulerian_cycle[1:end_index+1]
      else:
        if (end_index - start_index + 1) == len(eulerian_cycle):
          return eulerian_cycle[start_index:end_index+1]

In [291]:
def ReturnResult(eulerian_cycle):
  string_to_print = ''
  for node in eulerian_cycle:
    string_to_print = string_to_print + str(node) + '->'
  return string_to_print[0:len(string_to_print)-2]

In [292]:
def EulerianPath(graph_dict):
  unbalanced_nodes = FindUnbalancedNodes(graph_dict)
  unbalanced_nodes = UnbalancedNodesOrder(graph_dict,unbalanced_nodes)
  graph_dict = BalanceUnbalancedNodes(graph_dict,unbalanced_nodes)
  eulerian_cycle = EulerianCycle(graph_dict)
  eulerian_path = FindEulerianPathInEulerianCycle(unbalanced_nodes,eulerian_cycle)
  return eulerian_path

In [293]:
graph = [
'0 -> 2',
'1 -> 3',
'2 -> 1',
'3 -> 0,4',
'6 -> 3,7',
'7 -> 8',
'8 -> 9',
'9 -> 6']

In [294]:
graph

['0 -> 2',
 '1 -> 3',
 '2 -> 1',
 '3 -> 0,4',
 '6 -> 3,7',
 '7 -> 8',
 '8 -> 9',
 '9 -> 6']

In [295]:
graph_dict = FormatGraph(graph)

In [296]:
graph_dict

{0: [2], 1: [3], 2: [1], 3: [0, 4], 6: [3, 7], 7: [8], 8: [9], 9: [6]}

In [297]:
print(ReturnResult(EulerianPath(graph_dict)))

6->7->8->9->6->3->0->2->1->3->4


In [288]:
graph = []
with open('/content/rosalind_ba3g.txt') as task_file:
  graph = [line.rstrip() for line in task_file]

In [289]:
graph_dict = FormatGraph(graph)

In [ ]:
f = open("task_result.txt", "w")
f.write(ReturnResult(EulerianPath(graph_dict)))
f.close()